In [ ]:
# Install necessary libraries

!pip install opencv-python
!pip install roboflow
!pip install boto3 opencv-python-headless

In [ ]:
# Import required packages

import cv2
from roboflow import Roboflow
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
import boto3
import re

In [ ]:
# Convert .mkv file to mp4 in order to use cv2 package
!ffmpeg -i 1_224p.mkv -c copy output.mp4

In [ ]:
# Function to parse score from detected text
def parse_score(text):
    # Regular expression pattern for scores and time
    score_pattern = re.compile(r'\b(\d)-(\d)\b')  # Score pattern (e.g., "0-1")
    time_pattern = re.compile(r'\b(\d{2})-(\d{2})\b')  # Time pattern (e.g., "00-01")
    if time_pattern.match(text):
        return None  # It's a time, not a score
    score_match = score_pattern.search(text)
    if score_match:
        return score_match.group()  # It's a score

    return None

In [ ]:
# Function to check if the detected score is plausible based on the last known score

def is_plausible_score(last_score, new_score):
    if last_score is None:
        return True  # If there's no last score, the new score is plausible
    last_goals_home, last_goals_away = map(int, last_score.split('-'))
    new_goals_home, new_goals_away = map(int, new_score.split('-'))
    # Check if the difference in goals for either team is greater than one
    if last_goals_home - new_goals_home > 1 or last_goals_home - new_goals_home < 0  or last_goals_away - new_goals_away > 1 or last_goals_away - new_goals_away < 0:
        return False  # Implausible score change
    return True

In [ ]:
# Function to determine the color of a card in a region of interest (ROI) by creating masks for yellow and red colors
def detect_card_color(roi):
    # Convert ROI to HSV color space for better color detection
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define HSV color ranges for yellow and red colors
    yellow_lower, yellow_upper = np.array([15, 100, 100]), np.array([30, 255, 255])
    red_lower, red_upper = np.array([0, 100, 100]), np.array([20, 255, 255])

    # Create masks for detecting yellow and red colors within the defined ranges
    yellow_mask = cv2.inRange(hsv_roi, yellow_lower, yellow_upper)
    red_mask = cv2.inRange(hsv_roi, red_lower, red_upper)

    # Count the number of yellow and red pixels in the masks
    yellow_count, red_count = np.count_nonzero(yellow_mask), np.count_nonzero(red_mask)

    # Return 'yellow' if more yellow pixels are detected, otherwise 'red'
    return 'yellow' if yellow_count > red_count else 'red'


In [ ]:
# Function to process each frame.
# Functions include :
#                     1. Check for event of substitution.
#                     2. Check for event of booking and determine color of the card.
#                     2. Keep track of the scoreboard and update score.
def process_frame(frame, model, rekognition_client, last_known_score, frame_num, fps, process_scoreboard, last_booking_time, last_subs_time):

    # Calculate the current time in the video
    current_time = frame_num // fps

    # Get predictions from the model for the current frame
    inference_result1 = model.predict(frame, confidence=65, overlap=30).json()

    # Loop through each prediction in the frame
    for prediction in inference_result1['predictions']:
        # Process scoreboard updates
        if prediction['class'] == 'Scoreboards' and process_scoreboard:
            center_x, center_y, width, height = map(int, [prediction['x'], prediction['y'], prediction['width'], prediction['height']])
            top_left_x = max(center_x - width // 2, 0)
            top_left_y = max(center_y - height // 2, 0)
            roi = frame[top_left_y:top_left_y + height, top_left_x:top_left_x + width]


            conf = prediction['confidence']
            _, roi_buffer = cv2.imencode('.jpg', roi)
            detected_text_response = rekognition_client.detect_text(Image={'Bytes': roi_buffer.tobytes()})
            for item in detected_text_response['TextDetections']:
                detected_text = item['DetectedText']
                score = parse_score(detected_text)
                if score and is_plausible_score(last_known_score, score):
                    #cv2_imshow(roi)
                    minutes, seconds = divmod(current_time, 60)
                    minutes = int(minutes)
                    seconds = int(seconds)
                    if last_known_score != score:
                        print(f"Time: {minutes:02d}:{seconds:02d}, Score Change Detected: {score}, Frame: {frame_num},  confidence : {conf}")
                    last_known_score = score

        # Process booking events (like yellow or red cards)
        elif prediction['class'] == 'Booking' and prediction['confidence'] > 0.88:
            # Extract region of interest (ROI) for the scoreboard
            center_x, center_y, width, height = map(int, [prediction['x'], prediction['y'], prediction['width'], prediction['height']])
            top_left_x = max(center_x - width // 2, 0)
            top_left_y = max(center_y - height // 2, 0)
            roi = frame[top_left_y:top_left_y + height, top_left_x:top_left_x + width]

            # cv2_imshow(roi)
            conf = prediction['confidence']
            # Skip booking check if within 1.5 seconds of the last booking event
            if current_time - last_booking_time < 2.5:
                continue
            card_color = detect_card_color(roi)
            minutes, seconds = divmod(current_time, 60)
            minutes = int(minutes)
            seconds = int(seconds)
            print(f"Time: {minutes:02d}:{seconds:02d}, Booking Event Detected: {card_color} card, Frame: {frame_num}, confidence : {conf}")
            last_booking_time = current_time

        # Process substitution events
        elif prediction['class'] == 'Substitution' and prediction['confidence'] > 0.88:
            # Extract ROI for the substitution event
            center_x, center_y, width, height = map(int, [prediction['x'], prediction['y'], prediction['width'], prediction['height']])
            top_left_x = max(center_x - width // 2, 0)
            top_left_y = max(center_y - height // 2, 0)
            roi = frame[top_left_y:top_left_y + height, top_left_x:top_left_x + width]

            cv2_imshow(roi)
            conf = prediction['confidence']
            if current_time - last_subs_time < 2.5:
                continue
            minutes, seconds = divmod(current_time, 60)
            minutes = int(minutes)
            seconds = int(seconds)
            print(f"Time: {minutes:02d}:{seconds:02d}, Substitution Event Detected , Frame: {frame_num}, confidence : {conf}")
            last_booking_time = current_time

    # Return updated last_known_score and last_booking_time
    return last_known_score, last_booking_time


In [ ]:
def main():
    video_path = 'output.mp4'
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Roboflow and AWS Rekognition Initialization
    rf = Roboflow(api_key="")
    project = rf.workspace().project("scoreboarddetector")
    model = project.version(7).model
    aws_access_key_id = ''
    aws_secret_access_key = ''
    region_name = ''
    rekognition_client = boto3.client('rekognition', region_name=region_name, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
    frames_to_skip = 250
    last_known_score = None
    last_booking_time = -1.5  # Initialize to a value that allows immediate booking detection
    last_subs_time = -1.5
    frame_num = 0
    while cap.isOpened():
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        success, frame = cap.read()
        if not success:
            break

        process_scoreboard = frame_num % frames_to_skip == 0
        last_known_score, last_booking_time = process_frame(frame, model, rekognition_client, last_known_score, frame_num, fps, process_scoreboard, last_booking_time, last_subs_time)

        if frame_num % 1000 == 0:
          print("Current Frame:", frame_num)
        frame_num += 10
    cap.release()

if __name__ == "__main__":
    main()


loading Roboflow workspace...
loading Roboflow project...
Current Frame: 0
Time: 00:30, Score Change Detected: 0-0, Frame: 750,  confidence : 0.8783636093139648
Current Frame: 1000
Current Frame: 2000
Current Frame: 3000
Current Frame: 4000
Current Frame: 5000
Current Frame: 6000
Current Frame: 7000
Current Frame: 8000
Current Frame: 9000
Current Frame: 10000
Current Frame: 11000
Current Frame: 12000
Current Frame: 13000
Current Frame: 14000
Current Frame: 15000
Current Frame: 16000
Current Frame: 17000
Current Frame: 18000
Current Frame: 19000
Current Frame: 20000
Current Frame: 21000
Current Frame: 22000
Current Frame: 23000
Current Frame: 24000
Current Frame: 25000
Current Frame: 26000
Current Frame: 27000


Time: 18:20, Booking Event Detected: yellow card, Frame: 27500, confidence : 0.8983076214790344


Current Frame: 28000
Current Frame: 29000
Current Frame: 30000
Current Frame: 31000
Current Frame: 32000
Current Frame: 33000
Current Frame: 34000
Current Frame: 35000
Current Frame: 36000
Current Frame: 37000
Current Frame: 38000
Current Frame: 39000
Current Frame: 40000
Current Frame: 41000
Current Frame: 42000
Current Frame: 43000
Current Frame: 44000
Current Frame: 45000
Current Frame: 46000
Current Frame: 47000
Current Frame: 48000
Current Frame: 49000
Current Frame: 50000
Current Frame: 51000
Current Frame: 52000
Current Frame: 53000
Current Frame: 54000
Current Frame: 55000
Current Frame: 56000
Current Frame: 57000
Current Frame: 58000


KeyboardInterrupt: ignored